In [17]:
import utils

import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
from torch import optim
import time
import math

In [18]:
BS = 1000

In [19]:
train_iter, val_iter, test_iter, text_field = utils.load_imdb(batch_size=BS)

Loading Data
2
1


In [34]:
def neural_net_torch(nfeats,nclasses):
    model = torch.nn.Sequential()
    model.add_module('lin1',torch.nn.Linear(nfeats, 1000))
    model.add_module('ReLU',torch.nn.ReLU())
    model.add_module('lin2',torch.nn.Linear(1000,100))
    model.add_module('sigmoid',torch.nn.Sigmoid())
    model.add_module('lin3',torch.nn.Linear(100, nclasses))
    
    return model

def train(model,loss,optimizer,x,y):
    x = Variable(x)
    y = Variable(y)

    optimizer.zero_grad()

    forward_x = model.forward(x)
    obj = loss.forward(forward_x,y)

    obj.backward()

    optimizer.step()

    return obj.data[0]

def predict(model, x):
    # x = Variable(x)
    forward_x = model.forward(x)
    return forward_x.data.numpy().argmax(axis=1)

In [ ]:
eta = 0.05
model = neural_net_torch(245703,2)
optimizer = optim.SGD(model.parameters(),lr=eta)
loss = torch.nn.CrossEntropyLoss()

timebefore = time.clock()

print "TRAINING"

for k in range(0,30):
    cost = 0.
    for batch in train_iter:
        xval = utils.bag_of_words(batch,text_field)
        yval = batch.label-1
        cost += train(model,loss,optimizer,xval.data,yval.data)
    predtrain = predict(model,xval)
    print "Epoch %d, cost = %f, acc = %.2f%%"%(k + 1, cost, 100. * np.mean(predtrain == yval.data.numpy()))

    # Validation

    for batch in val_iter:
        xval = utils.bag_of_words(batch,text_field)
        yval1 = batch.label-1
    predval = predict(model,xval)
    print "Validation acc = %.2f%%"%(100. * np.mean(predval == yval1.data.numpy()))

print time.clock()-timebefore

TRAINING
Epoch 1, cost = 16.645996, acc = 51.20%
Validation acc = 52.80%
Epoch 2, cost = 16.552638, acc = 55.00%
Validation acc = 56.90%
Epoch 3, cost = 16.460857, acc = 57.80%
Validation acc = 59.00%
Epoch 4, cost = 16.344453, acc = 63.80%
Validation acc = 62.50%
Epoch 5, cost = 16.224695, acc = 66.70%
Validation acc = 60.60%
Epoch 6, cost = 16.042498, acc = 57.00%
Validation acc = 54.20%
Epoch 7, cost = 15.866636, acc = 64.80%
Validation acc = 65.40%
Epoch 8, cost = 15.698882, acc = 64.30%
Validation acc = 66.90%


In [ ]:
timebefore = time.clock()

print "TEST"

for batch in test_iter:
    xval = utils.bag_of_words(batch,text_field)
    yval = batch.label-1
predtest = predict(model,xval)
print "acc = %.2f%%"%(100. * np.mean(predtest == yval.data.numpy()))

print time.clock()-timebefore